In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 挑欄位作業

In [ ]:
pd.set_option("display.max_rows", 100)#設定讀取畫面範圍
pd.set_option("display.max_columns", 288)

In [ ]:
data=pd.read_csv('/kaggle/input/IMBDbigdata/0714train.csv')
data

In [ ]:
#以A:1,C:3為例子
col=[]
for i in range(1,25):
    col.append('Input_A1'+'_'+str(i).zfill(3))
for j in range(1,138):
    if(j%3!=0):
        col.append('Input_C_'+str(j).zfill(3))
data[col]

In [ ]:
def ch_col(data,a,c):#製造一個選欄位函式
    col=[]
    for i in range(1,25):
        col.append('Input_A'+str(a)+'_'+str(i).zfill(3))
    for j in range(1,138):
        if(j%c!=0):
            col.append('Input_C_'+str(j).zfill(3))
    return data[col]

In [ ]:
ch_col(data,1,3)#以A:1,C:3為例子

# 鐵達尼號提升準確率(更換各模型實測比較)

In [ ]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')
gender_submission=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [ ]:
train

In [ ]:
col=['Pclass','Embarked','SibSp','Parch']
data=pd.concat([train[col],test[col]],axis=0,ignore_index=True)
OH_data=pd.get_dummies(data,dummy_na=True, drop_first=False)#dummy_na將缺失值視為一個欄位
OH_data

In [ ]:
from sklearn.model_selection import train_test_split
X = OH_data[:891].copy()#前891筆 為訓練資料
y = train['Survived']#target
X_train, X_val, y_train, y_val =train_test_split(X,y, test_size=0.2,random_state=999)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
#以上都是分類器
from sklearn.metrics import accuracy_score
### 把模型存入串列
models= [   RandomForestClassifier(random_state=1),
			ExtraTreesClassifier(random_state=1),
			GradientBoostingClassifier(random_state=1),
            DecisionTreeClassifier(random_state=1),
            LGBMClassifier(random_state=1),
            XGBClassifier(random_state=1),
            AdaBoostClassifier(random_state=1),
            ##########以上全部為樹模型演算法
            SVC(),
            LogisticRegression(),
            KNeighborsClassifier(),
            LinearDiscriminantAnalysis(),
            GaussianNB(),
            Perceptron(),
            SGDClassifier(),
		]
for i,model in enumerate(models):#比較每個模型準確率
    print('第',i+1,'個模型')
    print("模型：{}".format(model))
    model.fit(X_train, y_train)
    predict = model.predict(X_val)
    print('acc:',accuracy_score(y_val,predict))#正確率

# 波士頓房價預測使用 GridSearchCV(自動調參） 

In [ ]:
house_data=pd.read_csv('../input/boston-housing-dataset/HousingData.csv')
house_data

In [ ]:
X=house_data.drop('MEDV',axis=1)
y=house_data['MEDV']#目標
X_train, X_val, y_train, y_val =train_test_split(X,y, test_size=0.2,random_state=999)

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
#XGB的參數

tree_param_grid= { 'max_depth':list(range(3,10,1)),#調參範圍
                   'min_child_weight':list(range(1,6,2))
                  }
grid=GridSearchCV(XGBRegressor( learning_rate= 0.1,# 如同學習率
                                #max_depth=4,# 構建樹的深度，越大越容易過擬合
                                colsample_bytree=1, # 生成樹時進行的列采樣
                                n_estimators=70, # 樹的個數
                              ),
                  param_grid=tree_param_grid,cv=10,#fold 數
                  scoring='neg_mean_squared_error')#評估方式
grid.fit(X_train,y_train)

In [ ]:
grid.cv_results_#運行結果

In [ ]:
grid.best_params_#最佳組合

In [ ]:
-grid.best_score_#最佳成績

In [ ]:
grid.best_estimator_#最佳模型

In [ ]:
best_model=grid.best_estimator_
predict=best_model.predict(X_val)
predict#預測結果

In [ ]:
from sklearn.metrics import mean_squared_error#評估模型誤差
from sklearn.metrics import mean_absolute_error

print('MAE:',mean_absolute_error(y_val,predict))
print('MSE:',mean_squared_error(y_val,predict))
print('RMSE:',mean_squared_error(y_val,predict)**0.5)